In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from time import sleep
from urllib.parse import urljoin

In [15]:
squad_urls = []
winners_list = []
years_list = []

wiki_url = "https://en.wikipedia.org/wiki/Category:FIFA_World_Cup-winning_players"
page = requests.get(wiki_url)
soup = BeautifulSoup(page.content, 'html.parser')
squad_url_links = soup.select("td ul li a")

for link in squad_url_links:
    years_list.append(link.text[:4])
    winners_list.append(link.text[6:])
    squad_urls.append(urljoin(wiki_url, link.get('href')))


In [16]:
country_list = []
player_list = []
age_list = []
club_list = []
time_list = []

for k, squad_url in enumerate(squad_urls):
    page = requests.get(squad_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    if squad_url in squad_urls[1:3]:
        team_names = soup.select("h2 span[class='mw-headline']")
    else:
        team_names = soup.select("h3 span[class='mw-headline']")
        if squad_url == squad_urls[13]:
            team_names.insert(5, team_names[4])
            team_names.insert(23, team_names[22])
            
    teams = soup.select("table[class='sortable wikitable plainrowheaders']")
    
    country_names = [x.text.strip() for x in team_names]
    winner_index = country_names.index(winners_list[k])
    
    
    team  = teams[winner_index]
    curr_country = winners_list[k]
    players = team.select("tbody tr")[1:]
    for player in players:
        country_list.append(curr_country)
        player_list.append(player.select("th")[0].text.strip())
        temp_age = player.select("td")[2].text.strip()
        if len(temp_age) > 8:
            age_list.append(temp_age[-3:-1])
        else:
            age_list.append('0')
        club_list.append(player.select("td")[-1].text.strip())
        time_list.append(years_list[k])
        
    
    sleep(5)

In [17]:
df = pd.DataFrame({'country':country_list, 'player':player_list, 'age':age_list, 'club':club_list, 'year':time_list})

In [40]:
import os

if not os.path.exists('./data'):
    os.mkdir('./data')

In [41]:
df.to_csv('data/winning_squads.csv', index=False)